# Parallel vs Sequential Tool Calls with Azure OpenAI Response API

This notebook demonstrates two different execution patterns:
1. **Parallel Tool Calls**: Multiple tools execute simultaneously (default behavior)
2. **Sequential Tool Calls**: Tools execute one after another in a specific order

Both patterns have their use cases depending on whether tool results depend on each other.

In [2]:
# Load environment variables
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Check if the API key is loaded
api_key = os.getenv("AZURE_OPENAI_API_KEY")
print(f"API Key loaded: {'✅ Yes' if api_key else '❌ No'}")
print(f"API Key (first 10 chars): {api_key[:10]}..." if api_key else "API Key: Not found")

# Show the base URL that will be used
base_url = "https://aifoundryarc.openai.azure.com/"
print(f"Base URL: {base_url}")

API Key loaded: ✅ Yes
API Key (first 10 chars): DXDrtFZ92E...
Base URL: https://aifoundryarc.openai.azure.com/


In [4]:
from openai import OpenAI

# Create the OpenAI client with Azure configuration
try:
    client = OpenAI(  
      base_url = "https://aifoundryarc.openai.azure.com/openai/v1",
      api_key=os.getenv("AZURE_OPENAI_API_KEY")  
    )
    print("✅ OpenAI client created successfully")
    
    # Make the response API call
    response = client.responses.create(
        input = "This is a test",
        model = "gpt-5-mini", # replace with model deployment name
        stream = True
    )
    
    print("🔄 Streaming response:")
    for event in response:
        if event.type == 'response.output_text.delta':
            print(event.delta, end='')
            
except Exception as e:
    print(f"❌ Error: {str(e)}")
    print(f"Error type: {type(e).__name__}")
    
    # Provide troubleshooting suggestions
    if "api_key" in str(e).lower():
        print("\n💡 Troubleshooting: API key issue")
        print("   - Make sure the .env file has AZURE_OPENAI_API_KEY set")
        print("   - Run the environment setup cell first")
    elif "model" in str(e).lower():
        print("\n💡 Troubleshooting: Model issue")
        print("   - Check if 'gpt-5-mini' is the correct model name")
        print("   - Verify the model is deployed in your Azure OpenAI resource")
    elif "endpoint" in str(e).lower() or "base_url" in str(e).lower():
        print("\n💡 Troubleshooting: Endpoint issue")
        print("   - Verify the base_url matches your Azure OpenAI endpoint")
        print("   - Check if the endpoint supports the responses API")
    else:
        print("\n💡 General troubleshooting:")
        print("   - Check your Azure OpenAI resource configuration")
        print("   - Verify the API version and model deployment")

✅ OpenAI client created successfully
🔄 Streaming response:
🔄 Streaming response:
Got it — test received. How can I help or what would you like to test next?Got it — test received. How can I help or what would you like to test next?

In [7]:
import os
from openai import OpenAI

client = OpenAI(  
  base_url = "https://aifoundryarc.openai.azure.com/openai/v1",
  api_key=os.getenv("AZURE_OPENAI_API_KEY")  
)

response = client.responses.create(  
    model="gpt-4o",  # replace with your model deployment name  
    tools=[  
        {  
            "type": "function",  
            "name": "get_weather",  
            "description": "Get the weather for a location",  
            "parameters": {  
                "type": "object",  
                "properties": {  
                    "location": {"type": "string"},  
                },  
                "required": ["location"],  
            },  
        }  
    ],  
    input=[{"role": "user", "content": "What's the weather in San Francisco and New York?"}],  
)  

print(response.model_dump_json(indent=2))  
  
# To provide output to tools, add a response for each tool call to an array passed  
# to the next response as `input`  
input = []  
for output in response.output:  
    if output.type == "function_call":  
        match output.name:  
            case "get_weather":  
                input.append(  
                    {  
                        "type": "function_call_output",  
                        "call_id": output.call_id,  
                        "output": '{"temperature": "70 degrees"}',  
                    }  
                )  
            case _:  
                raise ValueError(f"Unknown function call: {output.name}")  
  
second_response = client.responses.create(  
    model="gpt-4o",  
    previous_response_id=response.id,  
    input=input  
)  

print(second_response.model_dump_json(indent=2))

{
  "id": "resp_68d208e609f48193a34b1bcea1f4e225087032dd1462be23",
  "created_at": 1758595302.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-4o",
  "object": "response",
  "output": [
    {
      "arguments": "{\"location\":\"San Francisco\"}",
      "call_id": "call_rX4bQjg9YKUZbmNbVmVJxk6p",
      "name": "get_weather",
      "type": "function_call",
      "id": "fc_68d208e69b7481938d2049a726acb547087032dd1462be23",
      "status": "completed"
    },
    {
      "arguments": "{\"location\":\"New York\"}",
      "call_id": "call_UwlSjsF0UsKAxYZOa0Fkyik9",
      "name": "get_weather",
      "type": "function_call",
      "id": "fc_68d208e6bc6c8193a1ab6c0cebbe9521087032dd1462be23",
      "status": "completed"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [
    {
      "name": "get_weather",
      "parameters": {
        "type": "object",
        "properties": {
          "l

In [9]:
# Comprehensive Example: Parallel Tool Calls with Azure OpenAI Response API
import json
import time
from datetime import datetime

# Define tool functions that can be called in parallel
def get_weather(location):
    """Simulate weather API call"""
    time.sleep(0.5)  # Simulate API delay
    weather_data = {
        "San Francisco": {"temperature": "68°F", "condition": "Foggy", "humidity": "85%"},
        "New York": {"temperature": "72°F", "condition": "Sunny", "humidity": "60%"},
        "London": {"temperature": "55°F", "condition": "Rainy", "humidity": "90%"},
        "Tokyo": {"temperature": "75°F", "condition": "Cloudy", "humidity": "70%"}
    }
    return weather_data.get(location, {"temperature": "Unknown", "condition": "Data not available"})

def get_stock_price(symbol):
    """Simulate stock price API call"""
    time.sleep(0.3)  # Simulate API delay
    stock_data = {
        "AAPL": {"price": "$182.52", "change": "+1.25%", "volume": "45.2M"},
        "MSFT": {"price": "$378.85", "change": "+0.85%", "volume": "32.1M"},
        "GOOGL": {"price": "$142.36", "change": "-0.45%", "volume": "28.7M"},
        "TSLA": {"price": "$248.50", "change": "+2.15%", "volume": "67.3M"}
    }
    return stock_data.get(symbol, {"price": "Unknown", "change": "N/A", "volume": "N/A"})

def get_news_headlines(topic):
    """Simulate news API call"""
    time.sleep(0.4)  # Simulate API delay
    news_data = {
        "technology": [
            "AI breakthrough in quantum computing",
            "New smartphone features revolutionize mobile photography",
            "Tech giants announce major cloud infrastructure investments"
        ],
        "finance": [
            "Federal Reserve signals potential rate changes",
            "Cryptocurrency market shows strong recovery",
            "Global markets respond to economic indicators"
        ],
        "sports": [
            "Championship finals set record viewership",
            "Olympic preparations underway for next games",
            "Major league trades shake up team dynamics"
        ]
    }
    return news_data.get(topic, ["No news available for this topic"])

print("✅ Tool functions defined successfully!")
print("📋 Available tools:")
print("   - get_weather(location)")
print("   - get_stock_price(symbol)")
print("   - get_news_headlines(topic)")

✅ Tool functions defined successfully!
📋 Available tools:
   - get_weather(location)
   - get_stock_price(symbol)
   - get_news_headlines(topic)


In [12]:
# Parallel Tool Calls Example with Azure OpenAI Response API
try:
    client = OpenAI(  
        base_url = "https://aifoundryarc.openai.azure.com/openai/v1",
        api_key=os.getenv("AZURE_OPENAI_API_KEY")  
    )
    
    print("🚀 Making Response API call with parallel tool capabilities...")
    
    # Define tools for parallel execution
    tools = [
        {
            "type": "function",
            "name": "get_weather",
            "description": "Get current weather information for a specific location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city name to get weather for"
                    }
                },
                "required": ["location"]
            }
        },
        {
            "type": "function",
            "name": "get_stock_price",
            "description": "Get current stock price and trading information",
            "parameters": {
                "type": "object",
                "properties": {
                    "symbol": {
                        "type": "string",
                        "description": "The stock symbol (e.g., AAPL, MSFT)"
                    }
                },
                "required": ["symbol"]
            }
        },
        {
            "type": "function",
            "name": "get_news_headlines",
            "description": "Get latest news headlines for a specific topic",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "The news topic (e.g., technology, finance, sports)"
                    }
                },
                "required": ["topic"]
            }
        }
    ]
    
    # Make the initial response API call (parallel tool calls are enabled by default)
    response = client.responses.create(
        model="gpt-4o",
        tools=tools,
        input=[{
            "role": "user", 
            "content": "I need a quick update on: weather in San Francisco and New York, stock prices for AAPL and MSFT, and technology news headlines. Please get all this information for me."
        }]
        # Note: Parallel tool calls are enabled by default in the Response API
    )
    
    print("✅ Initial response received!")
    print(f"📊 Response ID: {response.id}")
    print(f"🔧 Tool calls detected: {len([output for output in response.output if output.type == 'function_call'])}")
    
    # Process tool calls and execute them
    tool_inputs = []
    start_time = time.time()
    
    for output in response.output:
        if output.type == "function_call":
            print(f"🔧 Executing tool: {output.name}")
            
            # Parse the function arguments
            args = json.loads(output.arguments) if hasattr(output, 'arguments') else {}
            
            # Execute the appropriate function
            if output.name == "get_weather":
                result = get_weather(args.get("location", ""))
            elif output.name == "get_stock_price":
                result = get_stock_price(args.get("symbol", ""))
            elif output.name == "get_news_headlines":
                result = get_news_headlines(args.get("topic", ""))
            else:
                result = {"error": f"Unknown function: {output.name}"}
            
            # Add the result to inputs for the next response
            tool_inputs.append({
                "type": "function_call_output",
                "call_id": output.call_id,
                "output": json.dumps(result)
            })
    
    execution_time = time.time() - start_time
    print(f"⏱️  Tool execution completed in {execution_time:.2f} seconds")
    
    # Get the final response with tool results
    print("\n📝 Getting final response with tool results...")
    final_response = client.responses.create(
        model="gpt-4o",
        previous_response_id=response.id,
        input=tool_inputs
    )
    
    # Display the final response
    print("\n🎯 Final Response:")
    print("=" * 60)
    for output in final_response.output:
        if output.type == "text":
            print(output.text)
    print("=" * 60)
    
    # Show execution summary
    print(f"\n📈 Execution Summary:")
    print(f"   • Tool calls executed: {len(tool_inputs)}")
    print(f"   • Total execution time: {execution_time:.2f} seconds")
    print(f"   • Parallel execution: {'✅ Enabled by default' if len(tool_inputs) > 1 else '❌ Single tool'}")
    
except Exception as e:
    print(f"❌ Error during parallel tool execution: {str(e)}")
    print(f"Error type: {type(e).__name__}")
    
    # Fallback: Demonstrate with mock response
    print("\n🎭 Demonstrating with mock parallel tool execution...")
    
    # Simulate parallel tool calls
    start_time = time.time()
    
    # These would normally be called in parallel by the API
    weather_sf = get_weather("San Francisco")
    weather_ny = get_weather("New York") 
    stock_aapl = get_stock_price("AAPL")
    stock_msft = get_stock_price("MSFT")
    tech_news = get_news_headlines("technology")
    
    execution_time = time.time() - start_time
    
    print(f"\n🎯 Mock Results (executed in {execution_time:.2f} seconds):")
    print("=" * 60)
    print(f"🌤️  San Francisco Weather: {weather_sf}")
    print(f"🌤️  New York Weather: {weather_ny}")
    print(f"📈 AAPL Stock: {stock_aapl}")
    print(f"📈 MSFT Stock: {stock_msft}")
    print(f"📰 Tech News: {tech_news}")
    print("=" * 60)

🚀 Making Response API call with parallel tool capabilities...
✅ Initial response received!
📊 Response ID: resp_68d20b22413081979239d0e0bf6961ea0f6443c280c9902a
🔧 Tool calls detected: 5
🔧 Executing tool: get_weather
✅ Initial response received!
📊 Response ID: resp_68d20b22413081979239d0e0bf6961ea0f6443c280c9902a
🔧 Tool calls detected: 5
🔧 Executing tool: get_weather
🔧 Executing tool: get_weather
🔧 Executing tool: get_weather
🔧 Executing tool: get_stock_price
🔧 Executing tool: get_stock_price
🔧 Executing tool: get_stock_price
🔧 Executing tool: get_stock_price
🔧 Executing tool: get_news_headlines
🔧 Executing tool: get_news_headlines
⏱️  Tool execution completed in 2.00 seconds

📝 Getting final response with tool results...
⏱️  Tool execution completed in 2.00 seconds

📝 Getting final response with tool results...

🎯 Final Response:

📈 Execution Summary:
   • Tool calls executed: 5
   • Total execution time: 2.00 seconds
   • Parallel execution: ✅ Enabled by default

🎯 Final Response:

📈 E

# Azure OpenAI Response API - Parallel Tool Calls

This example demonstrates how to use the Azure OpenAI Response API with parallel tool calls, which allows multiple functions to be executed concurrently for faster response times.

## Key Features:
- **Parallel Execution**: Multiple tools can run simultaneously
- **Structured Workflow**: Two-phase process (tool calls → results → final response)
- **Error Handling**: Graceful fallback with mock data
- **Performance Tracking**: Execution time monitoring

## Benefits of Parallel Tool Calls:
- **Faster Response**: Tools execute concurrently instead of sequentially
- **Better User Experience**: Reduced latency for multi-tool requests
- **Efficient Resource Usage**: Optimal utilization of API capabilities

In [ ]:
# Simplified Parallel Tool Call Example
print("🔧 Simplified Example: Two Tool Calls in Parallel")
print("=" * 50)

try:
    # Simple example with just 2 tools for clarity
    simple_tools = [
        {
            "type": "function",
            "name": "get_weather",
            "description": "Get weather for a location",
            "parameters": {
                "type": "object",
                "properties": {"location": {"type": "string"}},
                "required": ["location"]
            }
        },
        {
            "type": "function", 
            "name": "get_stock_price",
            "description": "Get stock price for a symbol",
            "parameters": {
                "type": "object",
                "properties": {"symbol": {"type": "string"}},
                "required": ["symbol"]
            }
        }
    ]
    
    # Make a simple request for 2 parallel operations
    # Note: Parallel tool calls are enabled by default in Response API
    simple_response = client.responses.create(
        model="gpt-4o",
        tools=simple_tools,
        input=[{"role": "user", "content": "Get weather for Tokyo and stock price for TSLA"}]
    )
    
    print("✅ Simple parallel request successful!")
    print(f"Tool calls in response: {len([o for o in simple_response.output if o.type == 'function_call'])}")
    
    # Execute the tools
    simple_inputs = []
    for output in simple_response.output:
        if output.type == "function_call":
            args = json.loads(output.arguments) if hasattr(output, 'arguments') else {}
            
            if output.name == "get_weather":
                result = get_weather(args.get("location", ""))
            elif output.name == "get_stock_price":
                result = get_stock_price(args.get("symbol", ""))
            
            simple_inputs.append({
                "type": "function_call_output",
                "call_id": output.call_id,
                "output": json.dumps(result)
            })
    
    # Get final response
    simple_final = client.responses.create(
        model="gpt-4o",
        previous_response_id=simple_response.id,
        input=simple_inputs
    )
    
    print("\n📋 Simple Response Result:")
    for output in simple_final.output:
        if output.type == "text":
            print(output.text)

except Exception as e:
    print(f"❌ Simple example failed: {str(e)}")
    
    # Mock demonstration
    print("\n🎭 Mock demonstration of parallel execution:")
    print("   ⚡ Tool 1: get_weather('Tokyo') → " + str(get_weather("Tokyo")))
    print("   ⚡ Tool 2: get_stock_price('TSLA') → " + str(get_stock_price("TSLA")))
    print("\n   💡 In the Response API, parallel tool calls are enabled by default!")
    print("   💡 Multiple tools in a single request will execute concurrently!")

🔧 Simplified Example: Two Tool Calls in Parallel
❌ Simple example failed: Responses.create() got an unexpected keyword argument 'response_config'

🎭 Mock demonstration of parallel execution:
   ⚡ Tool 1: get_weather('Tokyo') → {'temperature': '75°F', 'condition': 'Cloudy', 'humidity': '70%'}
   ⚡ Tool 1: get_weather('Tokyo') → {'temperature': '75°F', 'condition': 'Cloudy', 'humidity': '70%'}
   ⚡ Tool 2: get_stock_price('TSLA') → {'price': '$248.50', 'change': '+2.15%', 'volume': '67.3M'}

   💡 In real parallel execution, both tools would run simultaneously!
   ⚡ Tool 2: get_stock_price('TSLA') → {'price': '$248.50', 'change': '+2.15%', 'volume': '67.3M'}

   💡 In real parallel execution, both tools would run simultaneously!


In [13]:
# Sequential Tool Call Example - When Tools Depend on Each Other
print("🔄 Sequential Tool Call Example")
print("=" * 50)
print("Scenario: Get weather for a city, then get local news based on weather condition")

def get_local_news(location, weather_condition):
    """Get location-specific news based on weather condition"""
    time.sleep(0.3)
    
    weather_based_news = {
        "Sunny": [
            f"{location}: Perfect weather brings crowds to outdoor events",
            f"{location}: Solar energy production reaches peak efficiency",
            f"{location}: Beach tourism sees surge due to clear skies"
        ],
        "Rainy": [
            f"{location}: Umbrella sales spike as rain continues",
            f"{location}: Public transport delays due to weather conditions",
            f"{location}: Local farmers celebrate much-needed rainfall"
        ],
        "Foggy": [
            f"{location}: Airport delays expected due to reduced visibility",
            f"{location}: Fog creates mystical atmosphere for photographers",
            f"{location}: Maritime traffic slows in harbor"
        ],
        "Cloudy": [
            f"{location}: Overcast skies provide relief from summer heat",
            f"{location}: Outdoor concerts proceed despite cloud cover",
            f"{location}: Photographers capture dramatic cloud formations"
        ]
    }
    
    return weather_based_news.get(weather_condition, [f"{location}: Standard local news updates"])

def analyze_weather_trend(location, current_weather):
    """Analyze weather trend and provide recommendations"""
    time.sleep(0.4)
    
    recommendations = {
        "Sunny": f"Great day for outdoor activities in {location}! UV protection recommended.",
        "Rainy": f"Indoor activities recommended in {location}. Great day for museums or shopping.",
        "Foggy": f"Reduced visibility in {location}. Take extra care with transportation.",
        "Cloudy": f"Comfortable weather in {location}. Perfect for walking or light outdoor activities."
    }
    
    return {
        "recommendation": recommendations.get(current_weather, "Standard weather advisory"),
        "trend": "Stable conditions expected for next 6 hours",
        "air_quality": "Moderate to Good"
    }

try:
    print("\n🚀 Sequential Execution Pattern:")
    print("   Step 1: Get weather information")
    print("   Step 2: Use weather data to get relevant local news")
    print("   Step 3: Analyze weather trend with context")
    
    # Tool definitions for sequential calls
    weather_tool = {
        "type": "function",
        "name": "get_weather",
        "description": "Get current weather information for a location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string", "description": "City name"}
            },
            "required": ["location"]
        }
    }
    
    # STEP 1: Get weather information first
    print("\n🌤️  Step 1: Getting weather for San Francisco...")
    weather_response = client.responses.create(
        model="gpt-4o",
        tools=[weather_tool],
        input=[{"role": "user", "content": "What's the current weather in San Francisco?"}]
    )
    
    # Process weather result
    weather_result = None
    weather_inputs = []
    
    for output in weather_response.output:
        if output.type == "function_call" and output.name == "get_weather":
            args = json.loads(output.arguments) if hasattr(output, 'arguments') else {}
            weather_data = get_weather(args.get("location", ""))
            weather_result = weather_data
            
            weather_inputs.append({
                "type": "function_call_output",
                "call_id": output.call_id,
                "output": json.dumps(weather_data)
            })
    
    # Get weather summary
    weather_summary = client.responses.create(
        model="gpt-4o",
        previous_response_id=weather_response.id,
        input=weather_inputs
    )
    
    print("✅ Weather information received!")
    if weather_result:
        print(f"   Temperature: {weather_result.get('temperature', 'N/A')}")
        print(f"   Condition: {weather_result.get('condition', 'N/A')}")
    
    # STEP 2: Use weather data for local news (sequential dependency)
    if weather_result:
        print(f"\n📰 Step 2: Getting local news based on '{weather_result.get('condition', 'Unknown')}' weather...")
        
        local_news_tool = {
            "type": "function",
            "name": "get_local_news", 
            "description": "Get location-specific news based on weather condition",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string"},
                    "weather_condition": {"type": "string"}
                },
                "required": ["location", "weather_condition"]
            }
        }
        
        news_response = client.responses.create(
            model="gpt-4o",
            tools=[local_news_tool],
            input=[{
                "role": "user", 
                "content": f"Get local news for San Francisco considering the current weather is {weather_result.get('condition', 'Unknown')}"
            }]
        )
        
        # Process news result
        news_inputs = []
        news_result = None
        
        for output in news_response.output:
            if output.type == "function_call" and output.name == "get_local_news":
                args = json.loads(output.arguments) if hasattr(output, 'arguments') else {}
                news_data = get_local_news(
                    args.get("location", "San Francisco"),
                    args.get("weather_condition", weather_result.get('condition', 'Unknown'))
                )
                news_result = news_data
                
                news_inputs.append({
                    "type": "function_call_output",
                    "call_id": output.call_id,
                    "output": json.dumps(news_data)
                })
        
        print("✅ Local news retrieved based on weather!")
        
        # STEP 3: Analyze weather trend with context
        print(f"\n📊 Step 3: Analyzing weather trend...")
        
        trend_tool = {
            "type": "function",
            "name": "analyze_weather_trend",
            "description": "Analyze weather trend and provide recommendations",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string"},
                    "current_weather": {"type": "string"}
                },
                "required": ["location", "current_weather"]
            }
        }
        
        trend_response = client.responses.create(
            model="gpt-4o",
            tools=[trend_tool],
            input=[{
                "role": "user",
                "content": f"Analyze the weather trend for San Francisco with current condition: {weather_result.get('condition', 'Unknown')}"
            }]
        )
        
        # Process trend result
        trend_inputs = []
        
        for output in trend_response.output:
            if output.type == "function_call" and output.name == "analyze_weather_trend":
                args = json.loads(output.arguments) if hasattr(output, 'arguments') else {}
                trend_data = analyze_weather_trend(
                    args.get("location", "San Francisco"),
                    args.get("current_weather", weather_result.get('condition', 'Unknown'))
                )
                
                trend_inputs.append({
                    "type": "function_call_output",
                    "call_id": output.call_id,
                    "output": json.dumps(trend_data)
                })
        
        # Get final sequential response
        final_sequential = client.responses.create(
            model="gpt-4o",
            previous_response_id=trend_response.id,
            input=trend_inputs
        )
        
        print("\n🎯 Sequential Execution Results:")
        print("=" * 60)
        for output in final_sequential.output:
            if output.type == "text":
                print(output.text)
        print("=" * 60)
        
        print("\n📈 Sequential Execution Summary:")
        print("   • Step 1: Weather → Step 2: News → Step 3: Analysis")
        print("   • Each step depends on the previous step's results")
        print("   • Total steps: 3 (executed sequentially)")
        print("   • Use case: When tool outputs are interdependent")

except Exception as e:
    print(f"❌ Sequential execution error: {str(e)}")
    
    # Fallback demonstration
    print("\n🎭 Mock Sequential Execution:")
    print("   Step 1: Weather for San Francisco")
    weather_mock = get_weather("San Francisco")
    print(f"      Result: {weather_mock}")
    
    print(f"   Step 2: Local news based on '{weather_mock.get('condition')}' weather")
    news_mock = get_local_news("San Francisco", weather_mock.get('condition', 'Unknown'))
    print(f"      Result: {news_mock[0] if news_mock else 'No news'}")
    
    print(f"   Step 3: Weather trend analysis")
    trend_mock = analyze_weather_trend("San Francisco", weather_mock.get('condition', 'Unknown'))
    print(f"      Result: {trend_mock.get('recommendation', 'No recommendation')}")
    
    print("\n   💡 Sequential execution ensures each step has the data it needs!")

🔄 Sequential Tool Call Example
Scenario: Get weather for a city, then get local news based on weather condition

🚀 Sequential Execution Pattern:
   Step 1: Get weather information
   Step 2: Use weather data to get relevant local news
   Step 3: Analyze weather trend with context

🌤️  Step 1: Getting weather for San Francisco...
✅ Weather information received!
   Temperature: 68°F
   Condition: Foggy

📰 Step 2: Getting local news based on 'Foggy' weather...
✅ Weather information received!
   Temperature: 68°F
   Condition: Foggy

📰 Step 2: Getting local news based on 'Foggy' weather...
✅ Local news retrieved based on weather!

📊 Step 3: Analyzing weather trend...
✅ Local news retrieved based on weather!

📊 Step 3: Analyzing weather trend...

🎯 Sequential Execution Results:

📈 Sequential Execution Summary:
   • Step 1: Weather → Step 2: News → Step 3: Analysis
   • Each step depends on the previous step's results
   • Total steps: 3 (executed sequentially)
   • Use case: When tool outpu

In [14]:
# Comparison: Parallel vs Sequential Tool Execution
print("⚡ COMPARISON: Parallel vs Sequential Tool Execution")
print("=" * 70)

def time_execution(func, description):
    """Helper function to time execution"""
    start = time.time()
    result = func()
    elapsed = time.time() - start
    print(f"{description}: {elapsed:.2f} seconds")
    return result, elapsed

print("\n🏃‍♂️ PARALLEL EXECUTION SIMULATION:")
print("   All tools run simultaneously (fastest)")

def parallel_simulation():
    # Simulate parallel execution - all tools start at the same time
    start_time = time.time()
    
    # These would execute concurrently in the Response API
    weather_sf = get_weather("San Francisco")
    stock_aapl = get_stock_price("AAPL")  
    tech_news = get_news_headlines("technology")
    
    return {
        "weather": weather_sf,
        "stock": stock_aapl,
        "news": tech_news
    }

parallel_result, parallel_time = time_execution(parallel_simulation, "Parallel execution")

print("\n🚶‍♂️ SEQUENTIAL EXECUTION SIMULATION:")
print("   Tools run one after another (slower, but allows dependencies)")

def sequential_simulation():
    # Simulate sequential execution - tools run one at a time
    results = {}
    
    # Step 1: Get weather
    print("   → Step 1: Getting weather...")
    results["weather"] = get_weather("San Francisco")
    
    # Step 2: Use weather to determine appropriate stock sector
    weather_condition = results["weather"].get("condition", "").lower()
    if "sunny" in weather_condition or "clear" in weather_condition:
        stock_symbol = "AAPL"  # Tech stocks do well in good weather
        print("   → Step 2: Good weather detected, checking tech stocks...")
    else:
        stock_symbol = "MSFT"  # Different strategy for other weather
        print("   → Step 2: Other weather, checking different sector...")
    
    results["stock"] = get_stock_price(stock_symbol)
    
    # Step 3: Get news based on previous results
    if float(results["stock"]["price"].replace("$", "")) > 300:
        news_topic = "technology"
        print("   → Step 3: High stock price, getting tech news...")
    else:
        news_topic = "finance"
        print("   → Step 3: Lower stock price, getting finance news...")
    
    results["news"] = get_news_headlines(news_topic)
    
    return results

sequential_result, sequential_time = time_execution(sequential_simulation, "Sequential execution")

print(f"\n📊 PERFORMANCE COMPARISON:")
print(f"   ⚡ Parallel:   {parallel_time:.2f}s")
print(f"   🚶 Sequential: {sequential_time:.2f}s")
print(f"   🏆 Speed gain: {((sequential_time - parallel_time) / sequential_time * 100):.1f}% faster with parallel")

print(f"\n📋 WHEN TO USE EACH APPROACH:")
print(f"   🔄 PARALLEL (Default in Response API):")
print(f"      • Independent tool calls")
print(f"      • Maximum speed required")
print(f"      • Tools don't depend on each other's output")
print(f"      • Example: Get weather + stock + news simultaneously")

print(f"   ⏭️  SEQUENTIAL:")
print(f"      • Tool calls have dependencies")
print(f"      • Output of one tool affects another")
print(f"      • Logical workflow with steps")
print(f"      • Example: Weather → Weather-based recommendations → Local activities")

print(f"\n💡 RESPONSE API BEHAVIOR:")
print(f"   • Parallel execution is the DEFAULT behavior")
print(f"   • Multiple tools in one request = automatic parallel execution")
print(f"   • For sequential: make separate requests with previous results")
print(f"   • Use previous_response_id to chain requests together")

⚡ COMPARISON: Parallel vs Sequential Tool Execution

🏃‍♂️ PARALLEL EXECUTION SIMULATION:
   All tools run simultaneously (fastest)
Parallel execution: 1.20 seconds

🚶‍♂️ SEQUENTIAL EXECUTION SIMULATION:
   Tools run one after another (slower, but allows dependencies)
   → Step 1: Getting weather...
Parallel execution: 1.20 seconds

🚶‍♂️ SEQUENTIAL EXECUTION SIMULATION:
   Tools run one after another (slower, but allows dependencies)
   → Step 1: Getting weather...
   → Step 2: Other weather, checking different sector...
   → Step 2: Other weather, checking different sector...
   → Step 3: High stock price, getting tech news...
   → Step 3: High stock price, getting tech news...
Sequential execution: 1.20 seconds

📊 PERFORMANCE COMPARISON:
   ⚡ Parallel:   1.20s
   🚶 Sequential: 1.20s
   🏆 Speed gain: 0.1% faster with parallel

📋 WHEN TO USE EACH APPROACH:
   🔄 PARALLEL (Default in Response API):
      • Independent tool calls
      • Maximum speed required
      • Tools don't depend on

## Summary: Parallel vs Sequential Tool Calls

### Parallel Tool Calls (Default Behavior)
- **When to use**: Independent operations that don't depend on each other
- **Performance**: Fastest execution - all tools run simultaneously
- **Implementation**: Single Response API call with multiple tools
- **Example**: Get weather + stock prices + news headlines at the same time

### Sequential Tool Calls
- **When to use**: When tool outputs depend on previous results
- **Performance**: Slower but enables logical workflows
- **Implementation**: Multiple Response API calls using `previous_response_id`
- **Example**: Get weather → Get weather-based recommendations → Get local activities

### Best Practices
1. **Default to parallel** for maximum performance when possible
2. **Use sequential** only when tool outputs are interdependent
3. **Chain requests** using `previous_response_id` for sequential flows
4. **Consider hybrid approaches** - parallel within sequential steps